In [10]:
# นิสิตควรดำเนินการอย่างไรกับ outliners 
# ของการวิเคราะห์ความสัมพันธ์ CAPEX กับ ปีของข้อมูล

import pandas as pd
import numpy as np

dfComTH2 = pd.read_csv('DataComTH2.csv')
# dfComTH2.info()

# สำรวจลักษณะของ CAPEX ร่วมกับ ปี
# สิ่งที่มองหาคือ influence ของ outliners

dfCAPEX_Y = dfComTH2[['CAPEX','DataY']].groupby(by=['DataY']).mean().rename(columns={'CAPEX':'Mean'})
dfCAPEX_Y['Median'] = dfComTH2[['CAPEX','DataY']].groupby(by=['DataY']).median()
dfCAPEX_Y['Skew'] = dfComTH2[['CAPEX','DataY']].groupby(by=['DataY']).skew()
# print(dfComTH2['CAPEX'].skew())
# dfCAPEX_Y
dfComTH2['CAPEX-tx'] = dfComTH2['CAPEX'].apply(lambda x:np.log(x+0.5) if not pd.isna(x) else np.nan)

dfCAPEX_Y = dfComTH2[['CAPEX-tx','DataY']].groupby(by=['DataY']).mean().rename(columns={'CAPEX':'Mean'})
dfCAPEX_Y['Median'] = dfComTH2[['CAPEX-tx','DataY']].groupby(by=['DataY']).median()
dfCAPEX_Y['Skew'] = dfComTH2[['CAPEX-tx','DataY']].groupby(by=['DataY']).skew()
print(dfComTH2['CAPEX'].skew())
dfCAPEX_Y

# outliner influence is gone!

15.765107096919602


,CAPEX-tx,Median,Skew
DataY,,,
2000,4.121939,4.108388,0.249354
2001,4.038400,4.199991,-0.208074
2002,4.398797,4.376914,0.244943
2003,4.383430,4.328969,0.064981
2004,4.633081,4.727857,0.217096
2005,4.759438,4.689399,0.344552
2006,4.567571,4.517119,0.328268
2007,4.454225,4.389976,0.232112
2008,4.455205,4.337244,0.403204


In [22]:
# จำเป็นต้อง impute NumShares ควรดำเนินการอย่างไร

import pandas as np
import random

dfComTH2 = pd.read_csv('DataComTH2.csv')

# ศึกษา ลักษณะของ NumShares 
# เป็นข้อมูล (on the surface, how to read) ซึ่งขึ้นอยู่กับ บริษัท และ ปี

missingDict = {k:int(dfComTH2[dfComTH2['ComName']==k][['NumShares']].isna().sum()) for k in dfComTH2['ComName'].unique()}
dfMissing = pd.DataFrame(missingDict.items(),columns=['ComName','Missing'])
# comNameMissing = dfMissing.sort_values(by=['Missing'],ascending=True).head(15)['ComName']
# พบว่าปีแรกๆ มี numshares น้อยกว่าปีหลังๆ
comNameMissing = dfMissing.sort_values(by=['Missing'],ascending=False).head(20)['ComName']
# พบว่า missing มักเกิดขึ้นกับปีแรกๆ มีจำนวนน้อยมาก missing เกิดขึ้นในช่วงปีหลังๆ

for name in comNameMissing:
  mask = dfComTH2['ComName'] == name
  # print(dfComTH2[mask][['DataY','NumShares']])
    
filledNA = list()
for name in dfComTH2['ComName'].unique():
  filledNA.append(dfComTH2[dfComTH2['ComName']==name][['DataY','NumShares']].fillna(method='bfill'))

dfFilledNA = pd.concat(filledNA)
dfFilledNA.drop(columns=['DataY'],inplace=True)
dfFilledNA.columns = ['bFill']
dfComTH2 = dfComTH2.join(dfFilledNA)

filledNA = list()
for name in dfComTH2['ComName'].unique():
  filledNA.append(dfComTH2[dfComTH2['ComName']==name][['DataY','bFill']].fillna(method='ffill'))
dfFilledNA = pd.concat(filledNA)
dfFilledNA.drop(columns=['DataY'],inplace=True)
dfFilledNA.columns = ['Imputed']
dfComTH2 = dfComTH2.join(dfFilledNA)

comNames = random.choices(dfComTH2['ComName'].unique(),k=10)

# for name in comNameMissing:
for name in comNames:
  mask = dfComTH2['ComName'] == name
  print(dfComTH2[mask][['DataY','NumShares','bFill','Imputed']])


      DataY  NumShares    bFill  Imputed
4482   2000        NaN  119.000  119.000
4483   2002    119.000  119.000  119.000
4484   2003    149.621  149.621  149.621
4485   2004    182.782  182.782  182.782
4486   2005    183.830  183.830  183.830
4487   2006    224.840  224.840  224.840
4488   2007    234.810  234.810  234.810
4489   2008    234.810  234.810  234.810
4490   2009    234.810  234.810  234.810
4491   2010    234.810  234.810  234.810
4492   2011    234.810  234.810  234.810
4493   2012    234.810  234.810  234.810
4494   2013    258.291  258.291  258.291
4495   2014    258.291  258.291  258.291
4496   2015    258.291  258.291  258.291
4497   2016    258.291  258.291  258.291
4498   2017    258.291  258.291  258.291
4499   2018    258.291  258.291  258.291
4500   2019    258.291  258.291  258.291
4501   2020    258.291  258.291  258.291
4502   2021    258.291  258.291  258.291
      DataY  NumShares    bFill  Imputed
9183   2013        NaN  653.151  653.151
9184   2014     

In [38]:
from sklearn.impute import KNNImputer
import pandas as pd

dfComTH2 = pd.read_csv('DataComTH2.csv')

dfComTH2KNN = dfComTH2.drop(columns=['DataD','ComName','GlobID','NAICS']).copy()
KNNImp = KNNImputer(n_neighbors=5)
dfMissKNN = pd.DataFrame(KNNImp.fit_transform(dfComTH2KNN),columns=dfComTH2KNN.columns)
dfMissKNN = dfMissKNN.join(dfComTH2[['ComName']])
              
comNames = random.choices(dfComTH2['ComName'].unique(),k=10)

# for name in comNames:
#   mask = dfMissKNN['ComName'] == name
#   print(dfMissKNN[mask][['DataY','NumShares']])

mask = dfMissKNN['ComName'] == 'SCG CERAMICS PCL'

dfMissKNN = dfMissKNN.rename(columns={'NumShares':'SharesImp'})
dfMissKNN = dfMissKNN.join(dfComTH2['NumShares'])
dfMissKNN[mask][['DataY','NumShares','SharesImp']]
# dfMissKNN.info()

,DataY,NumShares,SharesImp
219,2000.0,NaN,157.3976
220,2001.0,NaN,611.0964
221,2002.0,NaN,2751.9284
222,2003.0,NaN,1607.0588
223,2004.0,NaN,2532.8710
224,2005.0,NaN,1324.0284
225,2006.0,NaN,1804.5246
226,2007.0,NaN,2797.6978
227,2008.0,NaN,80.5488
228,2009.0,NaN,258.9436


<font size='+2'><u>Week-11</u></font><br>
HTML scraping -> beautifulsoup<br>
NLP (natural language processing) pipeline<br>
.Thai NLP<br>
. X X X<br>
.find similar article -> cosine simlarity<br>
.topic modeling -> LDA (Latent Dirichlet Allocation) (week-12)<br>

In [44]:
import pandas as pd
from bs4 import BeautifulSoup

# ขั้นตอนการทำงาน
# 1. อ่าน file HTML เข้า 'bs4'
# 2. หา element ใน HTML ที่ต้องการ โดย bs4 object
# 3. extract ข้อมูล ของ element นั้น

htmlFile = open('blognone.html','r')
htmlDoc = htmlFile.read()
htmlFile.close()
bs4doc = BeautifulSoup(htmlDoc,'html.parser')
# print(bs4doc.prettify())

articles = list()
articleBoxes = bs4doc.find_all('div',class_='node clearfix')
for box in articleBoxes:
  # print(box.prettify())
  article = dict()
  articleTitle = box.find('a') # หา tag <a> ตัวแรก
  article['title'] = articleTitle.attrs['title']
  articleContent = box.find_all('p')
  articleText = ''
  for p in articleContent:
    articleText += p.text
  article['text'] = articleText
  articles.append(article)
  # print(article['title'])
  # print(article['text'])
  # print('=====')

dfArticles = pd.DataFrame(articles)
display(dfArticles)
dfArticles.to_json('blognone.json',orient='table')

,title,text
0,ร้อนแรงดั่งไฟเออร์ ผู้ใช้ GeForce 4090 รายงานป...,มีผู้ใช้การ์ดจอ NVIDIA GeForce RTX 4090 รุ่นให...
1,"ผู้บริหาร Apple ระบุ ""จะทำตามกฎหมาย EU"" แต่ไม่...",หลัง EU ผ่านกฎหมายให้อุปกรณ์อิเล็กทรอนิกส์ต้อง...
2,ฟีเจอร์ SFTP บน Azure Blob Storage เข้าสู่สถาน...,เมื่อเดือนพฤศจิกายน 2021 Microsoft ได้ประกาศเพ...
3,Meta เปิดตัวเซิร์ฟเวอร์รุ่นใหม่ Grand Teton ใช...,ในงานประชุม Open Compute Global Summit 2022 เม...
4,Stripe ผู้ให้บริการโซลูชั่นการชำระเงิน เปิดให้...,Stripe สตาร์ทอัพที่ให้บริการโซลูชั่นการชำระเงิ...
5,ซัมซุงเริ่มปล่อยฟีเจอร์โหมดส่งเครื่องซ่อม Main...,ซัมซุงประกาศปล่อยฟีเจอร์ Maintenance Mode ที่ช...
6,สิ้นสุดยุคขิปขาดแคลน Texas Instrument ระบุตลาด...,วันนี้บริษัทเซมิคอนดักเตอร์รายสำคัญอย่าง Texas...
7,Google Home ออกเวอร์ชันเว็บ ยังดูได้เฉพาะภาพจา...,กูเกิลออกไคลเอนต์ของแอพ Google Home เวอร์ชันเว...
8,Linus เสนอให้เคอร์เนลลินุกซ์หยุดซัพพอร์ตซีพียู...,Linus Torvalds แสดงความเห็นเรื่องสถาปัตยกรรมซี...
9,Unity ประกาศจะออกเอนจินเกมเวอร์ชัน Windows on Arm,เอนจินเกมชื่อดัง Unity ประกาศรองรับ Windows on...


In [46]:
# unzip

import zipfile as zf

filePath = 'bkknews/{0}'
newsZF = zf.ZipFile(filePath.format('bkknews.zip'),mode='r')
# print(filePath.format('bkknews.zip'))

for file in newsZF.namelist():
  newsFile = newsZF.open(file)
  fileHTML = open(filePath.format(file),mode='w',encoding='utf-8')
  fileHTML.write(str(newsFile.read(),'utf-8'))
  fileHTML.close()
newsZF.close()
print('DONE')


DONE


In [57]:
import os
import pandas as pd
from bs4 import BeautifulSoup

filePath = 'bkknews'
fileList = os.listdir(filePath)
fileList = [f for f in fileList if f[-4:] == 'html']

htmlFile = open(filePath+'/{0}'.format(fileList[1]),'r')
htmlDoc = htmlFile.read()
htmlFile.close()
bs4doc = BeautifulSoup(htmlDoc,'html.parser')
# print(bs4doc.prettify())

contentBoxes = bs4doc.find_all('div',class_='content-detail')
articleText = ''
for box in contentBoxes:
  articleContent = box.find_all('p')
  for p in articleContent:
    articleText += p.text

articleText = articleText.replace('\n\n\n','')
print(articleText)

นายสมพงษ์ รุ่งนิรัติศัย ประธานคณะผู้บริหาร ธุรกิจโลตัส ประเทศไทย กล่าวว่า พื้นฐานที่สำคัญของการเป็นผู้นำ New SMART Retail คือ นโยบายและแผนงานด้านความยั่งยืน โดย โลตัส ได้ดำเนินงานตามแนวทางด้านความยั่งยืนมาโดยตลอด และด้วยสถานการณ์ด้านสิ่งแวดล้อม สังคม และเศรษฐกิจที่เปลี่ยนแปลงไปอย่างมากในช่วงที่ผ่านมา โลตัส จึงได้ทบทวนและยกระดับการดำเนินงานด้านความยั่งยืน สู่นโยบายที่เราประกาศในวันนี้ คือ Vision 2030. Actions every day. ที่มุ่งเน้นการสร้างความยั่งยืนจากการดำเนินงานในทุกๆ วัน เพื่อให้บรรลุตามเป้าหมายที่ได้วางไว้ภายในปี ค.ศ. 2030 นโยบายด้านความยั่งยืนของ โลตัส สอดคล้องกับเป้าหมายด้านความยั่งยืนทั้ง 15 ข้อภายใต้ เครือเจริญโภคภัณฑ์ โดยจากการประเมินประเด็นสำคัญด้านความยั่งยืน (Materiality Assessment) ล่าสุด เราได้กำหนดประเด็นสำคัญ 5 ด้าน ที่ครอบคลุมทั้งด้านสิ่งแวดล้อม สังคม และการกำกับดูแลกิจการ (ESG) คือโดยมีการกำหนดยุทธศาสตร์และแผนงานที่ชัดเจนสำหรับแต่ละประเด็นสำคัญ เพื่อให้องค์กรสามารถบรรลุเป้าหมายที่วางไว้ภายในปี ค.ศ. 2030ประเด็นด้านสังคม (Social)ในฐานะธุรกิจค้าปลีกแบบ omni-channel ที่จำ

In [68]:
import os
import pandas as pd
from bs4 import BeautifulSoup

filePath = 'bkknews'
fileList = os.listdir(filePath)
fileList = [f for f in fileList if f[-4:] == 'html']

articles = dict()
articles['file'] = list()
articles['text'] = list()

for file in fileList:
  htmlFile = open(filePath+'/{0}'.format(file),'r')
  htmlDoc = htmlFile.read()
  htmlFile.close()
  bs4doc = BeautifulSoup(htmlDoc,'html.parser')
# print(bs4doc.prettify())

  contentBoxes = bs4doc.find_all('div',class_='content-detail')
  articleText = ''
  for box in contentBoxes:
    articleContent = box.find_all('p')
    for p in articleContent:
      articleText += p.text

  articleText = articleText.replace('\n\n\n','')
  articleText = articleText.replace('\xa0','')
  if len(articleText) > 10:
    articles['file'].append(file)
    articles['text'].append(articleText)
    
    
dfArticles = pd.DataFrame(articles)
dfArticles.to_json('bkknews.json',orient='table')
print('DONE')

DONE


<font size='+2'>NLP data prep pipeline</font><br>
1. การแยกคำ (tokenization)<br>
2. การหาคำร่วม (N-Grams)<br>
3. stopword (คำซึ่งไม่มีประโยชน์) (week-12)<br>
4. การสร้าง "vector" ข้อมูลการเกิดขึ้นของคำ (vectorization)(week-12)<br>

อะไรคือวัตถุประสงค์ของ pipline นี้ <br>
extract ข้อมูล และ จัดระเบียบข้อมูล <br>

ข้อมูลคืออะไร<br>
1. การมีอยู่ของคำ (bag of words)<br>
2. การมีอยู่ของคำ และ ความสำคัญของคำในประโยค (word embedding)<br>



In [73]:
from pythainlp import tokenize # dictionary based + heuristic based
from pythainlp.corpus.common import thai_words
from pythainlp.util import dict_trie
import pandas as pd

news = pd.read_json('bkknews.json',orient='table')

# news.info()

dictTH = set(thai_words())
dictTH = dict_trie(dict_source=dictTH)

words=list()
for text in news['text']:
  words.append([t for t in tokenize.word_tokenize(text,custom_dict=dictTH) if len(t) > 2])
  # print(w)

news['words'] = words
news

,file,text,words
0,1034093.html,ผู้สื่อข่าวจังหวัดปราจีนบุรีรายงานว่า พบอดีตห...,"[ผู้สื่อข่าว, จังหวัด, ปราจีนบุรี, รายงาน, ว่า..."
1,1034191.html,นายสมพงษ์ รุ่งนิรัติศัย ประธานคณะผู้บริหาร ธุร...,"[นาย, พงษ์, รุ่ง, นิรัติศัย, ประธาน, คณะ, ผู้บ..."
2,1034272.html,บริษัท ซีพีเอส ชิปปิ้ง แอนด์ โลจิสติกส์ จำกัดห...,"[บริษัท, พีเอส, ชิป, ปิ้ง, แอนด์, โลจิสติกส์, ..."
3,1034280.html,คณบดีคณะนิติศาสตร์ มหาวิทยาลัยรังสิตและประธานม...,"[คณบดี, คณะนิติศาสตร์, มหาวิทยาลัยรังสิต, และ,..."
4,1034281.html,มาตรการช่วย ‘ลูกหนี้’ น้ำท่วม ร่วมพูดคุยกับคุณ...,"[มาตรการ, ช่วย, ลูกหนี้, น้ำท่วม, ร่วม, พูดคุย..."
5,1034282.html,สถานการณ์น้ำท่วมในพื้นที่อำเภอชุมพวง จังหวัดนค...,"[สถานการณ์, น้ำท่วม, พื้นที่, อำเภอ, ชุม, พวง,..."
6,1034284.html,ผู้สื่อข่าวลงพื้นที่สำรวจราคาจำหน่ายผักสดภายใน...,"[ผู้สื่อข่าว, พื้นที่, สำรวจ, ราคา, จำหน่าย, ผ..."
7,1034285.html,สถานการณ์น้ำท่วม ในพื้นที่จังหวัดพิจิตร โดยเฉ...,"[สถานการณ์, น้ำท่วม, พื้นที่, จังหวัด, พิจิตร,..."
8,1034289.html,พร้อมฟังมุมมองภาคอสังหา​ริมทรัพย์​ไทย​ เห็นตรง...,"[พร้อม, ฟัง, มุมมอง, ภาค, อสังหา, ริม, ทรัพย์,..."
9,1034290.html,รองปลัดกระทรวงการคลัง รักษาราชการแทน อธิบดีกรม...,"[รอง, ปลัด, กระทรวงการคลัง, รักษา, ราชการ, แทน..."


In [79]:
# หาคำร่วม N-Grams
# คำใด มักจะปรากฎอยู่ด้วยกัน

from nltk.util import ngrams
from collections import Counter

allWords = list()
for words in news['words']:
  allWords += words

# len(allWords)
biGrams = ngrams(allWords,2)
wordsBiGrams = Counter(biGrams)
print(wordsBiGrams.most_common(10))
print()
triGrams = ngrams(allWords,3)
wordsTriGrams = Counter(triGrams)
print(wordsTriGrams.most_common(20))

[(('ประเทศ', 'ไทย'), 15), (('ล้าน', 'บาท'), 13), (('ซึ่ง', 'เป็น'), 12), (('ของ', 'โลตัส'), 11), (('อย่าง', 'ต่อเนื่อง'), 10), (('ของ', 'ประเทศ'), 10), (('ซื้อ', 'ราคา'), 10), (('ราคา', 'น้ำมัน'), 10), (('ภายใน', 'ค.ศ.'), 9), (('การ', 'ใช้'), 9)]

[(('ภายใน', 'ค.ศ.', '2030'), 6), (('บริษัท', 'พีเอส', 'ชิป'), 5), (('พีเอส', 'ชิป', 'ปิ้ง'), 5), (('ชิป', 'ปิ้ง', 'แอนด์'), 5), (('ปิ้ง', 'แอนด์', 'โลจิสติกส์'), 5), (('แอนด์', 'โลจิสติกส์', 'จำกัด'), 5), (('โลจิสติกส์', 'แบบ', 'ครบวงจร'), 5), (('การ', 'ให้', 'ความช่วยเหลือ'), 5), (('กิโลกรัม', 'บาท', 'ปัจจุบัน'), 5), (('ได้มา', 'ซึ่ง', 'ที่ดิน'), 5), (('ซึ่ง', 'ที่ดิน', 'เพื่อ'), 5), (('ที่ดิน', 'เพื่อ', 'ใช้'), 5), (('เพื่อ', 'ใช้', 'เป็น'), 5), (('ใช้', 'เป็น', 'ที่อยู่อาศัย'), 5), (('ราคา', 'น้ำมัน', 'พรุ่งนี้'), 5), (('ราคา', 'ทอง', 'วันนี้'), 5), (('สินค้า', 'แบรนด์', 'ของ'), 4), (('แบรนด์', 'ของ', 'โลตัส'), 4), (('การ', 'ปล่อย', 'ก๊าซ'), 4), (('ปล่อย', 'ก๊าซ', 'เรือนกระจก'), 4)]


In [84]:
# นำ n-gram มาเป็น "custom dict"

from pythainlp import tokenize # dictionary based + heuristic based
from pythainlp.corpus.common import thai_words
from pythainlp.util import dict_trie
import pandas as pd

news = pd.read_json('bkknews.json',orient='table')

# news.info()

dictTH = set(thai_words())
for gramWords,gramCount in wordsBiGrams.most_common(10):
  dictTH.add(''.join(gramWords))
dictTH.add('สุพัฒนพงษ์')
dictTH.add('พันธ์มีเชาว์')
dictTH.add('พลบุตร')
dictTH = dict_trie(dict_source=dictTH)

words=list()
for text in news['text']:
  words.append([t for t in tokenize.word_tokenize(text,custom_dict=dictTH) if len(t) > 2])
  # print(w)

news['words'] = words
news.to_json('bkknews.json',orient='table')
print('DONE')

DONE
